In [4]:
# Install keras_squeezenet
!pip install keras-squeezenet

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-squeezenet: filename=keras_squeezenet-0.4-py3-none-any.whl size=3577 sha256=19081aae0e0d67968989519e6b475ea2264561349d936469c7029ed435f7ab37
  Stored in directory: c:\users\bismoy\appdata\local\pip\cache\wheels\c5\8b\bb\eff0cb7dd0853f712e4ace15fbc569abd6141321ded205abdf
Successfully built keras-squeezenet


In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet, VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, Input, Conv2D, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, precision_score, recall_score

In [7]:
base_dir = 'E:/Undergrad Research/BanglaDatasetUS/'
train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')
models_dir = 'E:/Undergrad Research/BanglaDatasetUS/'

In [8]:
# Ensure the models directory exists
os.makedirs(models_dir, exist_ok=True)

In [9]:
# Image data generators
img_size = (224, 224)
batch_size = 32

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 2304 images belonging to 36 classes.


In [12]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 576 images belonging to 36 classes.


In [13]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 720 images belonging to 36 classes.


In [14]:
# Function to create SqueezeNet model manually
def SqueezeNet(input_shape=(224, 224, 3), classes=1000):
    input_img = Input(shape=input_shape)
    
    x = Conv2D(96, (7, 7), strides=(2, 2), padding='valid', activation='relu', name='conv1')(input_img)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1')(x)
    
    def fire_module(x, fire_id, squeeze=16, expand=64):
        s_id = 'fire' + str(fire_id) + '/'
        x = Conv2D(squeeze, (1, 1), padding='valid', activation='relu', name=s_id + 'squeeze1x1')(x)
        left = Conv2D(expand, (1, 1), padding='valid', activation='relu', name=s_id + 'expand1x1')(x)
        right = Conv2D(expand, (3, 3), padding='same', activation='relu', name=s_id + 'expand3x3')(x)
        x = concatenate([left, right], axis=3, name=s_id + 'concat')
        return x

    x = fire_module(x, fire_id=2)
    x = fire_module(x, fire_id=3)
    x = fire_module(x, fire_id=4)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool4')(x)
    
    x = fire_module(x, fire_id=5)
    x = fire_module(x, fire_id=6)
    x = fire_module(x, fire_id=7)
    x = fire_module(x, fire_id=8)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool8')(x)
    
    x = fire_module(x, fire_id=9)
    x = Dropout(0.5, name='drop9')(x)
    
    x = Conv2D(classes, (1, 1), padding='valid', name='conv10')(x)
    x = GlobalAveragePooling2D(name='avgpool10')(x)
    x = Dense(classes, activation='softmax', name='softmax')(x)
    
    model = Model(input_img, x, name='squeezenet')
    return model

In [15]:
# Function to create model
def create_model(base_model):
    base_model.trainable = False  # Freeze the base model initially
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),  # Add dropout to reduce overfitting
        Dense(train_generator.num_classes, activation='softmax')
    ])
    return model

In [16]:
# List of models
models = {
    "MobileNet": MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "SqueezeNet": SqueezeNet(input_shape=(224, 224, 3), classes=train_generator.num_classes),
    "VGG16": VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "ResNet50": ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    "EfficientNetB0": EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
}

In [17]:
results = []

In [18]:
# Callbacks for fine-tuning
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

In [ ]:
# Train and evaluate each model
for model_name, base_model in models.items():
    print(f"Training {model_name}...")
    if model_name == "SqueezeNet":
        model = base_model  # Use the manually defined SqueezeNet model directly
    else:
        model = create_model(base_model)
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model with frozen base layers
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=10,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    if model_name != "SqueezeNet":
        # Unfreeze some top layers of the base model
        base_model.trainable = True
        for layer in base_model.layers[:100]:
            layer.trainable = False
    
        # Recompile the model with a lower learning rate for fine-tuning
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
        # Fine-tune the model
        history = model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=10,
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )
    
    # Save the trained model
    model_path = os.path.join(models_dir, f'{model_name}.h5')
    model.save(model_path)
    print(f"Saved {model_name} model to {model_path}")
    
    test_loss, test_acc = model.evaluate(test_generator, verbose=1)
    
    Y_pred = model.predict(test_generator)
    y_pred = np.argmax(Y_pred, axis=1)
    
    print(f"Classification Report for {model_name}:")
    print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))
    
    results.append({
        "Model": model_name,
        "Accuracy": test_acc,
        "Precision": precision_score(test_generator.classes, y_pred, average='weighted'),
        "Recall": recall_score(test_generator.classes, y_pred, average='weighted')
    })

Training MobileNet...
Epoch 1/10
72/72 [==============================] - 399s 6s/step - loss: 1.9455 - accuracy: 0.5234 - val_loss: 0.2805 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/10
72/72 [==============================] - 390s 5s/step - loss: 0.3538 - accuracy: 0.9436 - val_loss: 0.0586 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/10
72/72 [==============================] - 390s 5s/step - loss: 0.1500 - accuracy: 0.9774 - val_loss: 0.0209 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/10
72/72 [==============================] - 390s 5s/step - loss: 0.0871 - accuracy: 0.9909 - val_loss: 0.0143 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/10
72/72 [==============================] - 392s 5s/step - loss: 0.0515 - accuracy: 0.9961 - val_loss: 0.0115 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/10
72/72 [==============================] - 390s 5s/step - loss: 0.0481 - accuracy: 0.9948 - val_loss: 0.0161 - val_accuracy: 0.9931 - lr: 0.0010
Epoch 7/10
72/72 [==============================] - 38

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
# Plot the results
results_df.set_index('Model', inplace=True)
results_df.plot(kind='bar', figsize=(10, 6))
plt.title('Comparison of Pre-trained Models on Alphabet and Number Recognition')
plt.ylabel('Score')
plt.xlabel('Model')
plt.legend(loc='upper right')
plt.show()